In [1]:
!yarn application -list

2025-07-20 13:09:03,870 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at cluster-3609-m.us-central1-c.c.fluid-shadow-459720-q0.internal./10.128.0.12:8032
2025-07-20 13:09:04,257 INFO client.AHSProxy: Connecting to Application History server at cluster-3609-m.us-central1-c.c.fluid-shadow-459720-q0.internal./10.128.0.12:10200
Total number of applications (application-types: [], states: [SUBMITTED, ACCEPTED, RUNNING] and tags: []):1
                Application-Id	    Application-Name	    Application-Type	      User	     Queue	             State	       Final-State	       Progress	                       Tracking-URL
application_1753015901820_0002	        PySparkShell	               SPARK	      root	   default	           RUNNING	         UNDEFINED	            10%	http://cluster-3609-m.us-central1-c.c.fluid-shadow-459720-q0.internal:35053


In [2]:
#yarn application -kill application_1753015901820_0001

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
# 🔧 SOLUTION 1: Check if Spark session exists, create if needed
def get_spark_session():
    try:
        # Try to get existing active session
        spark = SparkSession.getActiveSession()
        if spark is None:
            # Create new session if none exists
            spark = SparkSession.builder \
                .appName("Module-3") \
                .getOrCreate()
        return spark
    except:
        # If anything goes wrong, create fresh session
        spark = SparkSession.builder \
            .appName("Module-3") \
            .getOrCreate()
        return spark

In [5]:
# Get or create Spark session
spark = get_spark_session()

In [6]:
print("✅ Spark Session Status:")
print(f"   App Name: {spark.sparkContext.appName}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   Spark Version: {spark.version}")
print(f"   Active: {not spark.sparkContext._jsc.sc().isStopped()}")

✅ Spark Session Status:
   App Name: PySparkShell
   Master: yarn
   Spark Version: 3.5.3
   Active: True


In [7]:
# Define the base GCS bucket path
gcs_bucket_path = "gs://retail-order-data-bucket/output/"

In [8]:
# Read each Parquet file into a DataFrame
# Customers DataFrame
customers_df = spark.read.parquet(f"{gcs_bucket_path}customers_clean_df.parquet")
print("Customers DataFrame Schema:")
customers_df.printSchema()
print("Customers DataFrame Count:", customers_df.count())
customers_df.show(5) # Show first 5 rows

Customers DataFrame Schema:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



Customers DataFrame Count: 99441


+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 5 rows



In [9]:
# Geolocation DataFrame
geolocation_df = spark.read.parquet(f"{gcs_bucket_path}geolocation_clean_df.parquet/")
print("\nGeolocation DataFrame Schema:")
geolocation_df.printSchema()
print("Geolocation DataFrame Count:", geolocation_df.count())
geolocation_df.show(5)


Geolocation DataFrame Schema:
root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)

Geolocation DataFrame Count: 728444


+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1023|-23.542959247850415|-46.631904601159945|       sao paulo|               SP|
|                       1244|-23.550230577298162|-46.660698193865784|       sao paulo|               SP|
|                       1424| -23.56816235942637| -46.66117892867444|       sao paulo|               SP|
|                       1417|-23.561656060793965| -46.66621924231049|       sao paulo|               SP|
|                       1410| -23.56631504385709| -46.66444195322872|       sao paulo|               SP|
+---------------------------+-------------------+-------------------+----------------+-----------------+
only showing top 5 rows



In [10]:
# Order Items DataFrame
order_items_df = spark.read.parquet(f"{gcs_bucket_path}order_items_clean_df.parquet/")
print("\nOrder Items DataFrame Schema:")
order_items_df.printSchema()
print("Order Items DataFrame Count:", order_items_df.count())
order_items_df.show(5)


Order Items DataFrame Schema:
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: decimal(10,2) (nullable = true)
 |-- freight_value: decimal(10,2) (nullable = true)
 |-- total_value: decimal(11,2) (nullable = true)
 |-- shipping_date: date (nullable = true)
 |-- shipping_year: integer (nullable = true)
 |-- shipping_month: integer (nullable = true)

Order Items DataFrame Count: 112650


+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+-------------+-------------+--------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|total_value|shipping_date|shipping_year|shipping_month|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+-------------+-------------+--------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|      72.19|   2017-09-19|         2017|             9|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|     259.83|   2017-05-03|         2017|             5|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|     2

In [11]:
# Order Payments DataFrame
order_payments_df = spark.read.parquet(f"{gcs_bucket_path}order_payments_clean_df.parquet/")
print("\nOrder Payments DataFrame Schema:")
order_payments_df.printSchema()
print("Order Payments DataFrame Count:", order_payments_df.count())
order_payments_df.show(5)


Order Payments DataFrame Schema:
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: decimal(10,2) (nullable = true)
 |-- installment_value: decimal(10,2) (nullable = true)

Order Payments DataFrame Count: 103886
+--------------------+------------------+------------+--------------------+-------------+-----------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|installment_value|
+--------------------+------------------+------------+--------------------+-------------+-----------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|            12.42|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|            24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65

In [12]:
# Orders DataFrame
orders_df = spark.read.parquet(f"{gcs_bucket_path}orders_clean_df.parquet/")
print("\nOrders DataFrame Schema:")
orders_df.printSchema()
print("Orders DataFrame Count:", orders_df.count())
orders_df.show(5)


Orders DataFrame Schema:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

Orders DataFrame Count: 99441
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+--------------------------

In [13]:
# Sellers DataFrame
sellers_df = spark.read.parquet(f"{gcs_bucket_path}sellers_clean_df.parquet/")
print("\nSellers DataFrame Schema:")
sellers_df.printSchema()
print("Sellers DataFrame Count:", sellers_df.count())
sellers_df.show(5)



Sellers DataFrame Schema:
root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)

Sellers DataFrame Count: 3095
+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
+--------------------+----------------------+-----------------+------------+
only showing top 5 rows



In [14]:
# Order Reviews DataFrame
order_reviews_df = spark.read.parquet(f"{gcs_bucket_path}orders_reviews_clean_df.parquet/")
print("\nOrder Reviews DataFrame Schema:")
order_reviews_df.printSchema()
print("Order Reviews DataFrame Count:", order_reviews_df.count())
order_reviews_df.show(5)


Order Reviews DataFrame Schema:
root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)

Order Reviews DataFrame Count: 99224
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46

In [15]:
# Products DataFrame
products_df = spark.read.parquet(f"{gcs_bucket_path}products_clean_df.parquet/")
print("\nProducts DataFrame Schema:")
products_df.printSchema()
print("Products DataFrame Count:", products_df.count())
products_df.show(5)


Products DataFrame Schema:
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_weight_kg: decimal(10,3) (nullable = true)
 |-- product_category_clean: string (nullable = true)

Products DataFrame Count: 32951
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+-----------------+----------------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|pr

##  Customer Analysis

In [16]:
#Q1.1: Find the top 10 states with the highest number of customers
q = customers_df.groupBy("customer_state").agg(count("customer_id").alias("total_no_customer")).orderBy(col("total_no_customer").desc()).limit(10)
q.show()

+--------------+-----------------+
|customer_state|total_no_customer|
+--------------+-----------------+
|            SP|            41746|
|            RJ|            12852|
|            MG|            11635|
|            RS|             5466|
|            PR|             5045|
|            SC|             3637|
|            BA|             3380|
|            DF|             2140|
|            ES|             2033|
|            GO|             2020|
+--------------+-----------------+



In [17]:
#Q1.2: Calculate the average number of orders per customer
co_df = customers_df.join(orders_df,"customer_id","inner")
count_order = co_df.groupBy("customer_id").agg(count("order_id").alias("order_count"))
avg_order = count_order.agg(avg("order_count"))
avg_order.show()
count_order.show()

+----------------+
|avg(order_count)|
+----------------+
|             1.0|
+----------------+



+--------------------+-----------+
|         customer_id|order_count|
+--------------------+-----------+
|f1e46939e6408b3e6...|          1|
|90d7075599361b694...|          1|
|a340ce6c3570e68d4...|          1|
|b76530b7e66b27cd6...|          1|
|4d91a0aeb419c5f26...|          1|
|2aec499f94f5e8278...|          1|
|791fffae1e2c66693...|          1|
|0721e1c4b91bc6ded...|          1|
|d480546bdc6b03fca...|          1|
|6f8b4eeaba59ef3fa...|          1|
|c0fe0fbc24994167d...|          1|
|42da09831872a4ecc...|          1|
|2a8da2a879305eff0...|          1|
|452c0341844c3025a...|          1|
|c61b59e94df1163ca...|          1|
|5ff12b5d7429e2bfd...|          1|
|44e0884274cbedabc...|          1|
|d0615859a639a94c1...|          1|
|62189b05812bf34dc...|          1|
|aa62a16b10e3fb24e...|          1|
+--------------------+-----------+
only showing top 20 rows



In [18]:
#Q1.3: Find customers who have made more than 3 orders
co_df = customers_df.join(orders_df,"customer_id","inner")
count_order = co_df.groupBy("customer_id").agg(count("order_id").alias("order_count"))
count_order = count_order.filter(col("order_count")>=3)
count_order.show()



+-----------+-----------+
|customer_id|order_count|
+-----------+-----------+
+-----------+-----------+



## 2. Product Performance

In [19]:
# Q2.1: Identify the top 20 best-selling products by quantity
poi = products_df.join(order_items_df,"product_id","inner")
best = poi.groupBy("product_id").agg(count("order_item_id").alias("order_count")).orderBy(col("order_count").desc())
best.show(20,False)


+--------------------------------+-----------+
|product_id                      |order_count|
+--------------------------------+-----------+
|aca2eb7d00ea1a7b8ebd4e68314663af|527        |
|99a4788cb24856965c36a24e339b6058|488        |
|422879e10f46682990de24d770e7f83d|484        |
|389d119b48cf3043d311335e499d9c6b|392        |
|368c6c730842d78016ad823897a372db|388        |
|53759a2ecddad2bb87a079a1f1519f73|373        |
|d1c427060a0f73f6b889a5c7c61f2ac4|343        |
|53b36df67ebb7c41585e8d54d6772e08|323        |
|154e7e31ebfa092203795c972e5804a6|281        |
|3dd2a17168ec895c781a9191c1e95ad7|274        |
|2b4609f8948be18874494203496bc318|260        |
|7c1bd920dbdf22470b68bde975dd3ccf|231        |
|a62e25e09e05e6faf31d90c6ec1aa3d1|226        |
|5a848e4ab52fd5445cdc07aab1c40e48|197        |
|bb50f2e236e5eea0100680137654686c|195        |
|e0d64dcfaa3b6db5c54ca298ae101d05|194        |
|e53e557d5a159f5aa2c5e995dfdf244b|183        |
|42a2c92a0979a949ca4ea89ec5c7b934|183        |
|b532349fe46b

In [20]:
#Q2.2: Find the most expensive product in each category
from pyspark.sql import Window
poi = products_df.join(order_items_df,"product_id","inner")
r = poi.groupBy("product_category_name","product_id").agg(max("total_value").alias("max_value"))
# Define a window specification: partition by 'category', order by 'value'
window_spec = Window.partitionBy("product_category_name").orderBy(col("max_value").desc())
# Calculate row number within each category
r = r.withColumn("row_number",row_number().over(window_spec))
r = r.filter(col("row_number")==1)
r.show(10)

+---------------------+--------------------+---------+----------+
|product_category_name|          product_id|max_value|row_number|
+---------------------+--------------------+---------+----------+
|                 NULL|4c50dcc50f1512f46...|  4042.74|         1|
| agro industria e ...|2b69866f22de8dad6...|  3184.55|         1|
|            alimentos|21b4d9c7183dd6f20...|   285.73|         1|
|    alimentos bebidas|dbd8772797eb43bcb...|   720.77|         1|
|                artes|1bdf5e6731585cf01...|  6726.66|         1|
|   artes e artesanato|6bb18295cc019bf3b...|   314.32|         1|
|    artigos de festas|833fb04f7b1c82365...|   602.58|         1|
|     artigos de natal|fad983a47c56c7050...|   368.66|         1|
|                audio|9bfc55df037ce3ac0...|   636.98|         1|
|           automotivo|628b17449083fb346...|  2322.32|         1|
+---------------------+--------------------+---------+----------+
only showing top 10 rows



In [21]:
#Q2.3: Calculate the average product weight by category
aw = products_df.groupBy("product_category_name").agg(avg("product_weight_kg"))
aw.show()

+---------------------+----------------------+
|product_category_name|avg(product_weight_kg)|
+---------------------+----------------------+
| fashion bolsas e ...|             0.4267644|
|     artigos de natal|             1.8498154|
|                 NULL|             2.2507652|
|            cine foto|             0.7957857|
|      cama mesa banho|             2.4564051|
|      fraldas higiene|             1.0375000|
|           perfumaria|             0.5292569|
| agro industria e ...|             5.2634054|
|         climatizacao|             4.4599597|
|     fashion calcados|             1.0445087|
| utilidades domest...|             3.0207936|
|           la cuisine|             4.3500000|
|             pet shop|             3.0888567|
| portateis cozinha...|             2.5175000|
| sinalizacao e seg...|             2.9698172|
|               flores|             1.4035714|
| portateis casa fo...|             3.0715161|
| fashion roupa inf...|             0.3240000|
|     malas a

## 3. Order Statistics

In [22]:
#Q3.1: Calculate total revenue by month and year
rx = order_items_df.groupBy("shipping_month","shipping_year").agg(sum("total_value").alias("total_revenu")).orderBy(col("shipping_year").asc(),col("shipping_month").asc())
rx.show()

+--------------+-------------+------------+
|shipping_month|shipping_year|total_revenu|
+--------------+-------------+------------+
|             9|         2016|      218.52|
|            10|         2016|    56945.07|
|            12|         2016|       19.62|
|             1|         2017|    92198.41|
|             2|         2017|   282731.52|
|             3|         2017|   396542.39|
|             4|         2017|   352845.76|
|             5|         2017|   586221.79|
|             6|         2017|   543623.12|
|             7|         2017|   544850.17|
|             8|         2017|   656244.33|
|             9|         2017|   711614.67|
|            10|         2017|   760543.96|
|            11|         2017|  1027375.70|
|            12|         2017|  1045817.04|
|             1|         2018|   987044.01|
|             2|         2018|   958197.45|
|             3|         2018|  1209226.26|
|             4|         2018|  1116664.98|
|             5|         2018|  

In [23]:
#Q3.2: Find the distribution of order statuses
os = orders_df.groupBy("order_status").agg(count("order_id"))
os.show()

+------------+---------------+
|order_status|count(order_id)|
+------------+---------------+
|   delivered|          96478|
|    canceled|            625|
|     created|              5|
|     shipped|           1107|
|    approved|              2|
|  processing|            301|
|    invoiced|            314|
| unavailable|            609|
+------------+---------------+



In [24]:
#Q3.3: Calculate the average order value
aov = order_items_df.groupBy("order_id").agg(sum("total_value"))
av = aov.agg(avg("sum(total_value)"))
av.show()

+---------------------+
|avg(sum(total_value))|
+---------------------+
|           160.577638|
+---------------------+



## INTERMEDIATE JOINS & TRANSFORMATIONS

## Multi-Table Analysis

In [25]:
#Q4.1: Join orders with customers to find the top 5 cities by total order value
r = customers_df.join(orders_df,"customer_id","inner")
rx = r.join(order_items_df,"order_id","inner")
top = rx.groupBy("customer_city").agg(sum("total_value").alias("total_revenu")).orderBy(col("total_revenu").desc()).limit(5)
top.show()

+--------------+------------+
| customer_city|total_revenu|
+--------------+------------+
|     sao paulo|  2170227.12|
|rio de janeiro|  1154234.02|
|belo horizonte|   416733.39|
|      brasilia|   352305.14|
|      curitiba|   244739.87|
+--------------+------------+



In [26]:
#Q4.2: Create a complete order summary including customer details, payment info,and items
co = customers_df.join(orders_df,"customer_id","inner")
cooi = r.join(order_items_df,"order_id","inner")
cooiop = cooi.join(order_payments_df,"order_id","inner")
cooiop.show(5)


25/07/20 13:10:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+------------------------+-------------+--------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+-------------+-------------+--------------+------------------+------------+--------------------+-------------+-----------------+
|            order_id|         customer_id|  customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|total_value|shipping_date|shipping_year|shipping_month|payment_sequential|payment_type|payment_installments|payment_value|installment_v

In [27]:
#Q4.3: Find sellers who sold to customers in more than 5 different states
soi = sellers_df.join(order_items_df,"seller_id","inner")
soio = soi.join(orders_df,"order_id","inner")
x = soio.join(customers_df,"customer_id","inner")
y = x.groupBy("seller_id").agg(countDistinct("customer_state"))
y = y.filter(col("count(DISTINCT customer_state)")>5)
y.show()

+--------------------+------------------------------+
|           seller_id|count(DISTINCT customer_state)|
+--------------------+------------------------------+
|8e6cc767478edae94...|                            15|
|acadd4d36859671cb...|                             6|
|b76dba6c951ab00dc...|                            17|
|33cbbec1e7e1044aa...|                             7|
|7a67c85e85bb2ce85...|                            26|
|3d8fa2f5b647373c8...|                            10|
|d2374cbcbb3ca4ab1...|                            22|
|26d8a1c7c75d51304...|                             7|
|238fac594e170b59c...|                             7|
|f4a04f7be452aa3bb...|                             7|
|4200ddf1967c5e41e...|                             8|
|a254c682cc01e119f...|                             6|
|bf3c6d2a28b2b5501...|                            14|
|8b28d096634035667...|                            18|
|e59aa562b9f8076dd...|                            15|
|c7a0a13cbcf29d5ce...|      

## Payment Analysis

In [28]:
from pyspark.sql import Window
#Q5.1: Calculate the most popular payment method by state
op = orders_df.join(order_payments_df,"order_id","inner")
opc = op.join(customers_df,"customer_id","inner")
z = opc.groupBy("customer_state","payment_type").agg(count("payment_type").alias("fav")).orderBy(col("customer_state"),col("payment_type"))
window_spec = Window.partitionBy("customer_state").orderBy(col("fav").desc())
z = z.withColumn("row_number()",row_number().over(window_spec))
z = z.filter(col("row_number()")==1)
z.show()

+--------------+------------+-----+------------+
|customer_state|payment_type|  fav|row_number()|
+--------------+------------+-----+------------+
|            AC| credit_card|   61|           1|
|            AL| credit_card|  341|           1|
|            AM| credit_card|  124|           1|
|            AP| credit_card|   47|           1|
|            BA| credit_card| 2662|           1|
|            CE| credit_card| 1091|           1|
|            DF| credit_card| 1700|           1|
|            ES| credit_card| 1573|           1|
|            GO| credit_card| 1520|           1|
|            MA| credit_card|  535|           1|
|            MG| credit_card| 9070|           1|
|            MS| credit_card|  519|           1|
|            MT| credit_card|  659|           1|
|            PA| credit_card|  728|           1|
|            PB| credit_card|  428|           1|
|            PE| credit_card| 1334|           1|
|            PI| credit_card|  389|           1|
|            PR| cre

In [29]:
#Q5.2: Find orders with multiple payment methods and their success rates
op = orders_df.join(order_payments_df,"order_id","inner")
u = op.groupBy("order_id").agg(count("payment_type").alias("count_type"))
u = u.filter(col("count_type")>1)
xu = u.join(orders_df,"order_id","inner")
xu.select("order_id","count_type","order_status").show()
# now time to get success rate
a = xu.count()
b= xu.filter(col("order_status")== "delivered").count()
success_ratio = (b/a) * 100
print(f"success rates orders with multiple payment methods ",{success_ratio})


+--------------------+----------+------------+
|            order_id|count_type|order_status|
+--------------------+----------+------------+
|f44cb69655f8e4d13...|         2|   delivered|
|7a5472f7c8cecc2e1...|         2|   delivered|
|cd05a1fc2781f43f5...|         2|   delivered|
|8ca5bdac5ebe8f2d6...|         9|   delivered|
|54066aeaaf3ac32e7...|         2|   delivered|
|251f0a3981c4a8cb8...|         5|   delivered|
|3f4f6a378519479cb...|         2|   delivered|
|b1ce906877fd32ecc...|         2|   delivered|
|eb73628992e7d5099...|         2|   delivered|
|818877ced1be365d1...|         2|   delivered|
|f6ba6de181266b918...|         2|   delivered|
|0fc3e31f16aa4bc4b...|         2|   delivered|
|1798a9d3b896e39ea...|         2|   delivered|
|d8d126f05b944cb1a...|         2|   delivered|
|f1d6313ed30c82ba1...|         4|   delivered|
|a856b544fe56eea44...|         2|   delivered|
|681c2cfadee843332...|         2|   delivered|
|94dccbc39bc2673b3...|         2|   delivered|
|f8a8d05d951c

success rates orders with multiple payment methods  {97.09557581898008}


In [30]:
#Q5.3: Analyze payment installment patterns by order value ranges
op = orders_df.join(order_payments_df,"order_id","inner")
ov = op.groupBy("order_id").agg(sum("payment_value").alias("total_order_value"),sum("payment_installments").alias("total_installments_for_order"))
ov = ov.withColumn("value_range",
                   when(col("total_order_value") < 100, "low")
                   .when((col("total_order_value") >= 100) & (col("total_order_value") < 500), "medium") 
                   .when(col("total_order_value") >= 500, "high")
                   .otherwise("other_range")
                  )
ov.show()

# installments per range and orders n perticular range
ipr = ov.groupBy("value_range").agg(sum("total_installments_for_order").alias("total_inst_in_range"),count("order_id").alias("total_orders_in_range"))
ipr.show()

#average number of installments per order with in the range
aio = ov.groupBy("value_range").agg(avg("total_installments_for_order"))
aio.show()

+--------------------+-----------------+----------------------------+-----------+
|            order_id|total_order_value|total_installments_for_order|value_range|
+--------------------+-----------------+----------------------------+-----------+
|1c4a92d82c1b0dec1...|           249.05|                           1|     medium|
|28eaf054725f4dd3c...|            45.00|                           2|        low|
|78cd965d0bc0388d3...|           189.37|                           1|     medium|
|126f2d9c30f82426d...|           155.01|                           1|     medium|
|d6d7c431275f0029d...|           161.71|                           1|     medium|
|502d7a6ef832644f5...|            80.92|                           6|        low|
|a268b656281801419...|           297.64|                           1|     medium|
|61fc33807aeed929a...|           396.40|                          10|     medium|
|fed4ca34aeb50d89a...|           204.08|                           4|     medium|
|8d682b67c7d1f4f

+-----------+-------------------+---------------------+
|value_range|total_inst_in_range|total_orders_in_range|
+-----------+-------------------+---------------------+
|       high|              26437|                 4296|
|     medium|             173242|                47889|
|        low|              96744|                47255|
+-----------+-------------------+---------------------+



+-----------+---------------------------------+
|value_range|avg(total_installments_for_order)|
+-----------+---------------------------------+
|       high|                6.153864059590316|
|     medium|                3.617573973146234|
|        low|               2.0472754205904136|
+-----------+---------------------------------+



## Geographic Analysis

In [31]:
from pyspark.sql.functions import col, datediff, avg
#Q6.1: Calculate the average delivery time by state
oc = customers_df.join(orders_df,"customer_id","inner")
oc = oc.withColumn("delivery_days",datediff(col("order_delivered_customer_date"),col("order_purchase_timestamp")))
oc.show()
avg_days = oc.groupBy("customer_state").agg(avg("delivery_days").alias("average_delivery_days"))
avg_days.show()

+--------------------+--------------------+------------------------+--------------------+--------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|delivery_days|
+--------------------+--------------------+------------------------+--------------------+--------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+
|9ef432eb625129730...|7c396fd4830fd0422...|                    3149|           sao paulo|            SP|e481f51cbdc54678b...|   delivered| 

+--------------+---------------------+
|customer_state|average_delivery_days|
+--------------+---------------------+
|            MS|   15.544935805991441|
|            CE|   21.200156372165754|
|            MG|   11.946543372963452|
|            DF|   12.899038461538462|
|            RO|    19.28395061728395|
|            AM|   26.358620689655172|
|            MT|   18.003386004514674|
|            SP|    8.700530929744414|
|            PB|    20.38878143133462|
|            BA|   19.278562653562652|
|            SE|    21.46268656716418|
|            RJ|   15.237675058690197|
|            AC|                 21.0|
|            PR|   11.938045906967297|
|            AP|    27.17910447761194|
|            RR|   29.341463414634145|
|            TO|   17.598540145985403|
|            ES|   15.723809523809523|
|            AL|    24.50125944584383|
|            RN|   19.223628691983123|
+--------------+---------------------+
only showing top 20 rows



In [45]:
#Q6.2: Find the distance between seller and customer locations (using lat/lng)

# Step 1: Pre-process geolocation_df to get average lat/lng for each zip code prefix
# This handles cases where a zip code might have multiple lat/lng entries
avg_geo = geolocation_df.groupBy("geolocation_zip_code_prefix").agg(avg("geolocation_lat").alias("avg_lat"),avg("geolocation_lng").alias("avg_lng"))

# Step 2: Join sellers_df with the processed geolocation data
sg = sellers_df.join(avg_geo,col("geolocation_zip_code_prefix") == col("seller_zip_code_prefix"),"inner").select(
    col("seller_id"),
    col("seller_zip_code_prefix"),
    col("avg_lat").alias("seller_lat"),
    col("avg_lng").alias("seller_lng")
)

# Step 3: Join customers_df with the processed geolocation data
cg = customers_df.join(avg_geo,col("geolocation_zip_code_prefix") == col("customer_zip_code_prefix"),"inner").select(
    col("customer_id"),
    col("customer_zip_code_prefix"),
    col("avg_lat").alias("customer_lat"),
    col("avg_lng").alias("customer_lng")
    
)

# Step 4: Combine orders with order items, then link seller and customer geolocations
combined_df = orders_df.join(order_items_df, "order_id", "inner") \
                       .join(sg, "seller_id", "inner") \
                       .join(cg, "customer_id", "inner")

#combined_df.show() shows some null values

# Filter out rows with null coordinates if any were missed in joins
combined_df = combined_df.filter(
    col("seller_lat").isNotNull() & col("seller_lng").isNotNull() &
    col("customer_lat").isNotNull() & col("customer_lng").isNotNull()
)

# Convert degrees to radians
combined_df = combined_df.withColumn("seller_lat_rad", radians(col("seller_lat"))) \
                         .withColumn("seller_lon_rad", radians(col("seller_lng"))) \
                         .withColumn("customer_lat_rad", radians(col("customer_lat"))) \
                         .withColumn("customer_lon_rad", radians(col("customer_lng")))

# Calculate differences
combined_df = combined_df.withColumn("dlat", col("customer_lat_rad") - col("seller_lat_rad")) \
                         .withColumn("dlon", col("customer_lon_rad") - col("seller_lon_rad"))

# Apply Haversine formula parts
# 'a' part of Haversine
combined_df = combined_df.withColumn("a",
                                     sin(col("dlat") / 2)**2 + \
                                     cos(col("seller_lat_rad")) * cos(col("customer_lat_rad")) * \
                                     sin(col("dlon") / 2)**2
                                    )

# 'c' part of Haversine (angular distance in radians)
combined_df = combined_df.withColumn("c", 2 * atan2(sqrt(col("a")), sqrt(lit(1) - col("a"))))

# Earth's radius in kilometers (approximate)
R = 6371

# Final distance in kilometers
final_df = combined_df.withColumn("distance_km", lit(R) * col("c"))

# Show relevant columns (order_id, seller_id, customer_id, and the calculated distance)
final_df.select(
    "order_id",
    "seller_id",
    "customer_id",
    "seller_lat",
    "seller_lng",
    "customer_lat",
    "customer_lng",
    "distance_km"
).show()

+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|            order_id|           seller_id|         customer_id|         seller_lat|         seller_lng|       customer_lat|       customer_lng|       distance_km|
+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+
|e481f51cbdc54678b...|3504c0cb71d7fa48d...|9ef432eb625129730...|-23.680862316249442|  -46.4443108384268|-23.577621867663854|-46.587054702933266|18.526869053443914|
|53cdb2fc8bc7dce0b...|289cdb325fb7e7f89...|b0830fb4747a6c6d2...|-19.807884545156714| -43.98081789877275|-12.186877292980054|-44.540231823101585| 849.5204699192725|
|47770eb9100c2d0c4...|4869f7a5dfa277a7d...|41ce2a54c0b03bf34...| -21.36347334469748|-48.229587976936884|-16.745149541882597| -48.51478334802312|  514.407596192531|
|949d5b44dbf5de9

In [49]:
#Q6.3: Identify the most active shipping routes (seller_state -> customer_state)
combine_df = sellers_df.join(order_items_df,"seller_id","inner") \
             .join(orders_df,"order_id","inner") \
            .join(customers_df,"customer_id","inner")

# Group by seller_state and customer_state and count the occurrences
route_counts = combine_df.groupBy("seller_state", "customer_state").agg(
    count("order_id").alias("number_of_orders")
)
route_counts.show()

most_active = route_counts.orderBy(col("number_of_orders").desc()).limit(1)
most_active.show()


+------------+--------------+----------------+
|seller_state|customer_state|number_of_orders|
+------------+--------------+----------------+
|          PR|            RO|              18|
|          MA|            SE|               6|
|          RS|            AL|              10|
|          SE|            PR|               2|
|          ES|            PE|               6|
|          MA|            RJ|              42|
|          MA|            ES|              13|
|          SP|            MG|            8703|
|          MT|            PE|               6|
|          PE|            DF|              16|
|          RJ|            PR|             234|
|          SC|            SP|            1506|
|          SP|            TO|             229|
|          PR|            AL|              46|
|          MG|            BA|             430|
|          MG|            MG|            1709|
|          RJ|            CE|              59|
|          MT|            SP|              34|
|          MA

+------------+--------------+----------------+
|seller_state|customer_state|number_of_orders|
+------------+--------------+----------------+
|          SP|            SP|           36192|
+------------+--------------+----------------+



In [50]:
spark.stop()